In [ ]:

! pip install flask_wtf
! pip install flask_login
! pip install email_validator
!pip install datetime
!pip install virtualenv
from flask import Flask, render_template, redirect, url_for, session, flash
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from datetime import datetime
from flask_bcrypt import Bcrypt


app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'c1155c6a351e49eba15c00ce577b259e'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'


login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class Todo(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    content = db.Column(db.String(200), nullable=False)
    date_created = db.Column(db.DateTime, default=datetime.utcnow)

    def __repr__(self):
        return '<Task %r>' % self.id

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    notes = db.relationship('Note', backref='writer', lazy='dynamic')


class Note(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    note_body = db.Column(db.String(100))
    note_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")


class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")



@app.route("/")
def index():
    return render_template('index.html')

@app.route("/blog-single")
def blog():
    return render_template('blog-single.html')

@app.route("/profile")
def profile():
    return render_template('profile.html')

@app.route("/community")
def community():
    return render_template('community.html')

@app.route("/dianpu")
def dianpu():
    return render_template('dianpu.html')

@app.route("/script")
def script():
    return render_template('script.html')

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        username=form.username.data
        if username == "iamadmin":
            login_user(user)
            return redirect(url_for("login"))
        else:
            if user:
                if bcrypt.check_password_hash(user.password, form.password.data):
                    login_user(user)
                    flash("成功登录！")
                    return render_template('login.html')
    
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)

@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)

@app.route('/logout',methods=['GET','POST'])
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))

@app.route("/detail")
def detail():
    return render_template('detail.html')

@app.route("/script-detail")
def scriptdetail():
    return render_template('script-detail.html')

@app.route("/map")
def map():
    return render_template('map.html')

@app.route("/portfolio-details")
def portfolio():
    return render_template('portfolio-details.html')

if __name__ == '__main__':
    app.run('localhost', 8794)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:8794/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2021 20:42:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 20:42:52] "GET /assets/img/counters-bg.jpg HTTP/1.1" 404 -
127.0.0.1 - - [13/Dec/2021 20:42:54] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 20:42:54] "GET /assets/img/overlay-bg.jpg HTTP/1.1" 404 -
127.0.0.1 - - [13/Dec/2021 20:43:02] "POST /register HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 20:43:02] "POST /register HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 20:43:03] "GET /assets/img/overlay-bg.jpg HTTP/1.1" 404 -
